In [2]:
#!/usr/bin/env python3
"""
Ethical Hacking Toolkit - LightScanner
Integrates port scanning, subdomain enumeration, web analysis, and CVE suggestions
"""

import os
import sys
import subprocess
import json
import requests
import argparse
from datetime import datetime
import threading
import queue

class LightScanner:
    def __init__(self, target_domain, output_file=None):
        self.target_domain = target_domain
        self.output_file = output_file or f"scan_results_{target_domain}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
        self.results = {
            'target': target_domain,
            'timestamp': datetime.now().isoformat(),
            'subdomains': [],
            'ports': [],
            'web_services': [],
            'vulnerabilities': []
        }

    def run_command(self, command):
        """Execute shell command and return output"""
        try:
            result = subprocess.run(command, shell=True, capture_output=True, text=True, timeout=300)
            return result.stdout, result.stderr, result.returncode
        except subprocess.TimeoutExpired:
            return "", "Command timed out", 1
        except Exception as e:
            return "", str(e), 1

    def check_dependencies(self):
        """Check if required tools are installed"""
        tools = ['nmap', 'sublist3r', 'whatweb']
        missing = []

        for tool in tools:
            try:
                subprocess.run([tool, '--version'], capture_output=True, check=True)
            except (subprocess.CalledProcessError, FileNotFoundError):
                missing.append(tool)

        if missing:
            print(f"Missing dependencies: {', '.join(missing)}")
            print("Please install:")
            print("  nmap: sudo apt-get install nmap")
            print("  sublist3r: pip install sublist3r")
            print("  whatweb: sudo apt-get install whatweb")
            return False
        return True

    def subdomain_enumeration(self):
        """Enumerate subdomains using Sublist3r"""
        print(f"[+] Enumerating subdomains for {self.target_domain}")

        # Use sublist3r
        cmd = f"sublist3r -d {self.target_domain} -o /tmp/sublist3r_{self.target_domain}.txt"
        stdout, stderr, returncode = self.run_command(cmd)

        subdomains = []
        try:
            with open(f"/tmp/sublist3r_{self.target_domain}.txt", 'r') as f:
                subdomains = [line.strip() for line in f.readlines() if line.strip()]
        except FileNotFoundError:
            pass

        # Add some common subdomains as fallback
        common_subs = ['www', 'mail', 'ftp', 'admin', 'test', 'dev', 'api', 'blog']
        for sub in common_subs:
            subdomains.append(f"{sub}.{self.target_domain}")

        self.results['subdomains'] = list(set(subdomains))
        print(f"  Found {len(self.results['subdomains'])} subdomains")
        return self.results['subdomains']

    def port_scanning(self, subdomain):
        """Perform port scanning using nmap"""
        print(f"[+] Scanning ports for {subdomain}")

        # Quick scan for common ports
        cmd = f"nmap -sS -T4 -F {subdomain} --open"
        stdout, stderr, returncode = self.run_command(cmd)

        ports = []
        for line in stdout.split('\n'):
            if '/tcp' in line and 'open' in line:
                parts = line.split()
                if len(parts) >= 2:
                    port_info = parts[0].split('/')
                    if len(port_info) >= 2:
                        ports.append({
                            'port': port_info[0],
                            'protocol': port_info[1],
                            'service': parts[2] if len(parts) > 2 else 'unknown',
                            'state': 'open'
                        })

        return ports

    def web_analysis(self, subdomain):
        """Analyze web services using whatweb"""
        print(f"[+] Analyzing web service for {subdomain}")

        cmd = f"whatweb {subdomain} --color=never"
        stdout, stderr, returncode = self.run_command(cmd)

        web_info = {
            'target': subdomain,
            'technologies': [],
            'headers': {},
            'status': 'unknown'
        }

        # Parse whatweb output
        for line in stdout.split('\n'):
            if '200 OK' in line:
                web_info['status'] = '200 OK'
            if '[' in line and ']' in line:
                tech = line.split('[')[1].split(']')[0]
                web_info['technologies'].append(tech)

        return web_info

    def get_cve_suggestions(self, service, version=""):
        """Get CVE suggestions from Vulners API"""
        print(f"[+] Checking CVEs for {service} {version}")

        try:
            # Vulners API endpoint
            url = "https://vulners.com/api/v3/search/lucene/"
            query = f"product:{service}"
            if version:
                query += f" version:{version}"

            headers = {'User-Agent': 'LightScanner/1.0'}
            response = requests.get(url, params={'query': query}, headers=headers, timeout=10)

            if response.status_code == 200:
                data = response.json()
                vulnerabilities = []

                for item in data.get('data', {}).get('search', []):
                    if '_source' in item:
                        vuln = item['_source']
                        vulnerabilities.append({
                            'cve': vuln.get('id', 'N/A'),
                            'type': vuln.get('type', 'N/A'),
                            'title': vuln.get('title', 'N/A'),
                            'description': vuln.get('description', 'N/A')[:200] + '...',
                            'cvss': vuln.get('cvss', {}).get('score', 'N/A'),
                            'published': vuln.get('published', 'N/A')
                        })

                return vulnerabilities[:5]  # Return top 5 results

        except requests.RequestException:
            pass

        return []

    def scan_target(self):
        """Main scanning function"""
        print(f"[*] Starting scan for {self.target_domain}")
        print("[*]=" * 30)

        if not self.check_dependencies():
            return False

        # Step 1: Subdomain enumeration
        subdomains = self.subdomain_enumeration()

        # Step 2: Scan each subdomain
        for subdomain in subdomains[:3]:  # Limit to 3 subdomains for demo
            print(f"\n[*] Processing {subdomain}")

            # Port scanning
            ports = self.port_scanning(subdomain)
            self.results['ports'].extend(ports)

            # Check for web services (ports 80, 443, 8080, etc.)
            web_ports = [p for p in ports if p['port'] in ['80', '443', '8080', '8443']]

            if web_ports:
                # Web analysis
                web_info = self.web_analysis(subdomain)
                web_info['ports'] = [p['port'] for p in web_ports]
                self.results['web_services'].append(web_info)

                # Get CVE suggestions for detected technologies
                for tech in web_info['technologies']:
                    cves = self.get_cve_suggestions(tech)
                    for cve in cves:
                        cve['target'] = subdomain
                        cve['service'] = tech
                        self.results['vulnerabilities'].append(cve)

            # Get CVE suggestions for other services
            for port_info in ports:
                if port_info['port'] not in ['80', '443', '8080', '8443']:
                    cves = self.get_cve_suggestions(port_info['service'])
                    for cve in cves:
                        cve['target'] = subdomain
                        cve['service'] = port_info['service']
                        cve['port'] = port_info['port']
                        self.results['vulnerabilities'].append(cve)

        # Save results
        self.save_results()
        return True

    def save_results(self):
        """Save results to file"""
        with open(self.output_file, 'w') as f:
            f.write("LightScanner Results\n")
            f.write("=" * 50 + "\n\n")

            f.write(f"Target: {self.results['target']}\n")
            f.write(f"Scan Date: {self.results['timestamp']}\n\n")

            f.write("SUBDOMAINS:\n")
            f.write("-" * 20 + "\n")
            for sub in self.results['subdomains']:
                f.write(f"  {sub}\n")
            f.write("\n")

            f.write("OPEN PORTS:\n")
            f.write("-" * 20 + "\n")
            for port in self.results['ports']:
                f.write(f"  {port['port']}/{port['protocol']} - {port['service']} ({port['state']})\n")
            f.write("\n")

            f.write("WEB SERVICES:\n")
            f.write("-" * 20 + "\n")
            for web in self.results['web_services']:
                f.write(f"  {web['target']} (Ports: {', '.join(web['ports'])})\n")
                f.write(f"    Status: {web['status']}\n")
                f.write(f"    Technologies: {', '.join(web['technologies'])}\n")
            f.write("\n")

            f.write("VULNERABILITY SUGGESTIONS:\n")
            f.write("-" * 30 + "\n")
            for vuln in self.results['vulnerabilities']:
                f.write(f"  CVE: {vuln.get('cve', 'N/A')}\n")
                f.write(f"    Target: {vuln.get('target', 'N/A')}\n")
                f.write(f"    Service: {vuln.get('service', 'N/A')}\n")
                f.write(f"    CVSS: {vuln.get('cvss', 'N/A')}\n")
                f.write(f"    Title: {vuln.get('title', 'N/A')}\n")
                f.write(f"    Description: {vuln.get('description', 'N/A')}\n")
                f.write("\n")

        print(f"\n[+] Results saved to: {self.output_file}")

    def display_summary(self):
        """Display summary of results"""
        print("\n" + "="*60)
        print("SCAN SUMMARY")
        print("="*60)
        print(f"Target: {self.results['target']}")
        print(f"Subdomains found: {len(self.results['subdomains'])}")
        print(f"Open ports: {len(self.results['ports'])}")
        print(f"Web services: {len(self.results['web_services'])}")
        print(f"Vulnerability suggestions: {len(self.results['vulnerabilities'])}")

        if self.results['vulnerabilities']:
            print("\nTOP VULNERABILITY SUGGESTIONS:")
            for vuln in self.results['vulnerabilities'][:3]:
                print(f"  {vuln.get('cve')} - {vuln.get('title')} (CVSS: {vuln.get('cvss')})")

def main():
    parser = argparse.ArgumentParser(description="LightScanner - Ethical Hacking Toolkit")
    parser.add_argument("target", help="Target domain to scan")
    parser.add_argument("-o", "--output", help="Output file name")

    # Parse known arguments and ignore the rest
    args, unknown = parser.parse_known_args()

    if not args.target:
        parser.print_help()
        sys.exit(1)

    print("""
    SCANNER FOR
    Lightweight Ethical Hacking Toolkit
    """)

    scanner = LightScanner(args.target, args.output)

    try:
        if scanner.scan_target():
            scanner.display_summary()
        else:
            print("Scan failed due to missing dependencies")
    except KeyboardInterrupt:
        print("\n[!] Scan interrupted by user")
        sys.exit(1)
    except Exception as e:
        print(f"[!] Error during scan: {e}")
        sys.exit(1)

if __name__ == "__main__":
    main()


    SCANNER FOR
    Lightweight Ethical Hacking Toolkit
    
[*] Starting scan for /root/.local/share/jupyter/runtime/kernel-4baf18cf-8466-4c16-b904-88721e673906.json
[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=[*]=
Missing dependencies: nmap, sublist3r, whatweb
Please install:
  nmap: sudo apt-get install nmap
  sublist3r: pip install sublist3r
  whatweb: sudo apt-get install whatweb
Scan failed due to missing dependencies
